In [1]:
import pandas as pd
import json

with open('News_Category_Dataset_v2.json') as json_file:  
    data = json.load(json_file)

In [2]:
sample = [ x for x in data if x['category'] in ['HEALTHY LIVING','WORLD NEWS','COMEDY','SPORTS','BLACK VOICES'] ]

In [3]:
len(sample)

23458

In [4]:
from random import shuffle
shuffle(sample)

In [5]:
train = sample[:20000]
validation = sample[20000:23000]
test = sample[23000:]

In [6]:
d={'HEALTHY LIVING':1,'WORLD NEWS':2,'COMEDY':3,'SPORTS':4,'BLACK VOICES':0}

In [7]:

L=[]
k=[]
C=[]
for item in train:
    c=d[item['category']]
    C.append(c)
    exp=[item['headline'],item['short_description']]
    EXP=item['headline']+item['short_description']
    L.append(exp)
    k.append(EXP)

In [8]:
T=[]
C_T=[ ]
for item in test:
    c=d[item['category']]
    EXP=item['headline']+item['short_description']
    T.append(EXP)
    C_T.append(c)
    

In [9]:
train_df = pd.DataFrame(k)
test_df = pd.DataFrame(T)

In [10]:
train_df['category']=C
test_df['category']=C_T

In [11]:
train_df.columns=["sentence","category"]
test_df.columns=["sentence","category"]

In [12]:
test_df.head()

,sentence,category
0,'Late Night' Writer Expertly Schools John Kell...,3
1,Feeling Draped In An Overcast Of 'Diversity'Di...,0
2,4 Skin Issues That Can Be Solved By A Good Nig...,1
3,The U.S. Will Probably Have At Least One Zika ...,1
4,Trump’s Plan To Kill The Iran Deal? Outsourcin...,2


In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import keras.layers as layers

from collections import Counter
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense, Concatenate,Activation
from keras.models import Model

# from keras.utils import plot_mode
from keras.utils import to_categorical

C:\Users\Adil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# Reduce TensorFlow logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

# Instantiate the elmo model
elmo_module = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

# Initialize session
sess = tf.Session()
K.set_session(sess)

K.set_learning_phase(1)

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [15]:
# parameter of max word length
time_steps = 100
# building vocabulary from dataset
def build_vocabulary(sentence_list):
    unique_words = " ".join(sentence_list).strip().split()
    word_count = Counter(unique_words).most_common()
    vocabulary = {}
    for word, _ in word_count:
        vocabulary[word] = len(vocabulary)        

    return vocabulary


# Get vocabulary vectors from document list
# Vocabulary vector, Unknown word is 1 and padding is 0
# INPUT: raw sentence list
# OUTPUT: vocabulary vectors list
def get_voc_vec(document_list, vocabulary):    
    voc_ind_sentence_list = []
    for document in document_list:
        voc_idx_sentence = []
        word_list = document.split()
        
        for w in range(time_steps):
            if w < len(word_list):
                # pickup vocabulary id and convert unknown word into 1
                voc_idx_sentence.append(vocabulary.get(word_list[w], -1) + 2)
            else:
                # padding with 0
                voc_idx_sentence.append(0)
            
        voc_ind_sentence_list.append(voc_idx_sentence)
        
    return np.array(voc_ind_sentence_list)


vocabulary = build_vocabulary(train_df["sentence"])

In [16]:
# mini-batch generator
def batch_iter(data, labels, batch_size, shuffle=True):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    print("batch_size", batch_size)
    print("num_batches_per_epoch", num_batches_per_epoch)

    def data_generator():
        data_size = len(data)

        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                                
                X_voc = get_voc_vec(shuffled_data[start_index: end_index], vocabulary)
                                
                sentence_split_list = []
                sentence_split_length_list = []
            
                for sentence in shuffled_data[start_index: end_index]:    
                    sentence_split = sentence.split()
                    sentence_split_length = len(sentence_split)
                    sentence_split += ["NaN"] * (time_steps - sentence_split_length)
                    
                    sentence_split_list.append((" ").join(sentence_split))
                    sentence_split_length_list.append(sentence_split_length)
        
                X_elmo = np.array(sentence_split_list)

                X =  X_elmo
                y = shuffled_labels[start_index: end_index]
                
                yield X, y

    return num_batches_per_epoch, data_generator()

In [18]:
# embed elmo method
def make_elmo_embedding(x):
    embeddings = elmo_module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["elmo"]
    
    return embeddings

In [34]:
# elmo embedding dimension
elmo_dim = 1024

# Input Layers
#word_input = Input(shape=(None, ), dtype='int32')  # (batch_size, sent_length)
elmo_input = Input(shape=(None, ), dtype=tf.string)  # (batch_size, sent_length, elmo_size)

# Hidden Layers
#word_embedding = Embedding(input_dim=len(vocabulary), output_dim=128, mask_zero=True)(word_input)
elmo_embedding = layers.Lambda(make_elmo_embedding, output_shape=(None, elmo_dim))(elmo_input)
#word_embedding = Concatenate()([word_embedding, elmo_embedding])
word_embedding = BatchNormalization()(elmo_embedding)
x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(word_embedding)

# Output Layer

predict = Dense(units=5, activation='softmax')(x)


model = Model(inputs=[elmo_input], outputs=predict)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

#plot_model(model, to_file="model.png", show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
lambda_3 (Lambda)            (None, None, 1024)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 1024)        4096      
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 595,077
Trainable params: 593,029
Non-trainable params: 2,048
_________________________________________________________________


In [20]:
time_steps = 100
# Create datasets (Only take up to time_steps words for memory)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
train_text = np.array(train_text)
train_label = np.array(train_df['category'].tolist())

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:time_steps]) for t in test_text]
test_text = np.array(test_text)
test_label = np.array(test_df['category'].tolist())

In [21]:
# mini-batch size
batch_size = 32

train_steps, train_batches = batch_iter(train_text,
                                        np.array(train_df["category"]),
                                        batch_size)
valid_steps, valid_batches = batch_iter(test_text,
                                        np.array(test_df["category"]),
                                        batch_size)

batch_size 32
num_batches_per_epoch 625
batch_size 32
num_batches_per_epoch 15


In [33]:
from keras.callbacks import ModelCheckpoint
filepath="weights.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model.fit([train_text], batch_size=64, y=to_categorical(train_label), verbose=1, validation_split=0.10, 
          shuffle=True, epochs=10, callbacks=[checkpointer])

Train on 18000 samples, validate on 2000 samples
Epoch 1/10
   64/18000 [..............................] - ETA: 2:42:09 - loss: 1.5945 - acc: 0.2500

KeyboardInterrupt: 

In [22]:
logfile_path = './log'
tb_cb = TensorBoard(log_dir=logfile_path, histogram_freq=0)

history = model.fit_generator(train_batches, train_steps,
                              epochs=5, 
                              validation_data=valid_batches,
                              validation_steps=valid_steps,
                              callbacks=[tb_cb])

Epoch 1/5
625/625 [==============================] - 13844s 22s/step - loss: 3.1354 - acc: 0.2834 - val_loss: 2.2710 - val_acc: 0.3079
Epoch 2/5
625/625 [==============================] - 9910s 16s/step - loss: 2.2011 - acc: 0.1156 - val_loss: 2.0819 - val_acc: 0.0721
Epoch 3/5
625/625 [==============================] - 9810s 16s/step - loss: 2.0097 - acc: 0.1434 - val_loss: 1.7269 - val_acc: 0.2140
Epoch 4/5
625/625 [==============================] - 9660s 15s/step - loss: 1.5302 - acc: 0.2795 - val_loss: 1.4195 - val_acc: 0.3734
Epoch 5/5
625/625 [==============================] - 9593s 15s/step - loss: 1.2913 - acc: 0.3592 - val_loss: 1.3584 - val_acc: 0.3843


In [1]:
from azureml.core import Experiment, Workspace

ws = Workspace.create(name='nnworkspace',
                      subscription_id='26268c67-6923-4060-b75e-e672e201df32',
                      resource_group='nnproject',
                      create_resource_group=True,
                      location='eastus' 
                     )


Deploying AppInsights with name nnworkspinsightsf45ea59a.
Deployed AppInsights with name nnworkspinsightsf45ea59a.
Deploying KeyVault with name nnworkspkeyvault2eb76ef6.
Deploying StorageAccount with name nnworkspstorage0b992773b.
Deployed KeyVault with name nnworkspkeyvault2eb76ef6.
Deploying Workspace with name nnworkspace.
Deployed StorageAccount with name nnworkspstorage0b992773b.
Deployed Workspace with name nnworkspace.


In [2]:
# Create a new experiment in your workspace.
exp = Experiment(workspace=ws, name='exp2')

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-05-21T17:30:13.530000+00:00', 'errors': None, 'creationTime': '2019-05-21T17:29:37.016645+00:00', 'modifiedTime': '2019-05-21T17:30:32.392339+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

data_folder = os.path.join(os.getcwd(), 'newsdata')
os.makedirs(data_folder, exist_ok = True)

ds.upload(src_dir=data_folder, target_path='newsdata', overwrite=True, show_progress=True)

AzureBlob nnworkspstorage0b992773b azureml-blobstore-4f341f39-327b-4d16-ae7c-2140930849f1
Uploading C:\Users\Adil\Desktop\PROJECT\Untitled Folder\newsdata\News_Category_Dataset_v2.json
Uploaded C:\Users\Adil\Desktop\PROJECT\Untitled Folder\newsdata\News_Category_Dataset_v2.json, 1 files out of an estimated total of 1


NameError: name 'scripts_folder' is not defined

In [6]:
script_folder = os.path.join(os.getcwd(), 'scripts')
os.makedirs(script_folder, exist_ok = True)
ds.upload(src_dir=script_folder, target_path='scripts', overwrite=True, show_progress=True)

Uploading C:\Users\Adil\Desktop\PROJECT\Untitled Folder\scripts\train_elmo.py
Uploaded C:\Users\Adil\Desktop\PROJECT\Untitled Folder\scripts\train_elmo.py, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_02b459f809d34dbc8356ddb0a95cd9eb

In [9]:
from azureml.train.dnn import TensorFlow


# os.makedirs(data_folder, exist_ok = True)
est = TensorFlow(source_directory=script_folder,
                script_params=None,
                compute_target=compute_target,
                entry_script='train_elmo.py',
                conda_packages=['pandas', 'tensorflow','scikit-learn'],
                pip_packages=['keras','tensorflow-hub'],
                use_gpu = True)

framework_version is not specified, defaulting to version 1.13.


In [174]:
ws.delete(delete_dependent_resources=True)

In [10]:
run = exp.submit(config=est)

In [11]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
exp2,exp2_1558460117_560f57a8,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


In [13]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…